In [1]:
"""
Entrenamiento y evaluación de un clasificador SVM lineal.

Se implementa un modelo binario (OK vs DEFICIENT_RESPONSE) utilizando un
feature de texto que incorpora la categoría de la respuesta (PURPOSE o INTERACTIONS). El modelo se
entrena con una partición 80/20 y se evalúa mediante accuracy, matriz de
confusión y recall para la clase DEFICIENT_RESPONSE.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score

from google.colab import files

uploaded = files.upload()

csv_filename = list(uploaded.keys())[0]

# encode latin1 para el csv por el lenguaje (tildes en español)
df = pd.read_csv(csv_filename, encoding="latin1")

# Estas son validaciones para asegurar que el dataset que se suba contiene las columnas necesarias para la clasificación
required_cols = ["RESPONSE_CATEGORY_ID", "ANSWER_TEXT", "ERROR_SUBCATEGORY_ID"]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Faltan columnas en el CSV: {missing}. Columnas encontradas: {list(df.columns)}")

df = df.dropna(subset=["ANSWER_TEXT", "RESPONSE_CATEGORY_ID", "ERROR_SUBCATEGORY_ID"]).copy()

# Al inicio hubo problemas por espacios en los nombres de las clases (ERROR_SUCATEGORY_ID), esto elimina los espacios de los labels
df["ERROR_SUBCATEGORY_ID"] = df["ERROR_SUBCATEGORY_ID"].astype(str).str.strip()
df["RESPONSE_CATEGORY_ID"] = df["RESPONSE_CATEGORY_ID"].astype(str).str.strip()

df["text"] = "__CAT_" + df["RESPONSE_CATEGORY_ID"].str.upper() + "__ " + df["ANSWER_TEXT"].astype(str)

X = df["text"]
y = df["ERROR_SUBCATEGORY_ID"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

vectorizer = TfidfVectorizer(
    lowercase=True,
    strip_accents="unicode",
    ngram_range=(1, 2)
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

svm = LinearSVC(random_state=42, dual="auto")
svm.fit(X_train_vec, y_train)

y_pred = svm.predict(X_test_vec)

accuracy = accuracy_score(y_test, y_pred)

labels_order = ["OK", "DEFICIENT_RESPONSE"]
cm = confusion_matrix(y_test, y_pred, labels=labels_order)

# Recall para DEFICIENT_RESPONSE
recall_def = recall_score(y_test, y_pred, pos_label="DEFICIENT_RESPONSE")

print("----- RESULTADOS SVM Linear -----")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall (DEFICIENT_RESPONSE): {recall_def:.4f}")
print("\nMatriz de confusión:")


# Gráfico matriz de confusión
plt.figure(figsize=(6, 5))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    xticklabels=labels_order,
    yticklabels=labels_order,
    cbar=False
)
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de Confusión - SVM (Linear)")
plt.tight_layout()
plt.show()


IndexError: list index out of range

In [ ]:
import joblib

joblib.dump(vectorizer, "vectorizer.joblib")
joblib.dump(svm, "svm_model.joblib")


['svm_model.joblib']

In [ ]:
import sklearn
print(sklearn.__version__)


1.6.1
